# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 438000  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
#list(get_annual.columns.values)

In [12]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [13]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [14]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [15]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [17]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanCost($)"] + allAnnualData["HeatingCost($)"] + allAnnualData["CoolingCost($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData

In [18]:
#list(allAnnualData.columns.values)

---

## Establish Annual Baseline at Clear/Clear-VB0

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [19]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baselineAnnual

In [20]:
# Rename columns to indicate baseline to prepare for merge

allBaselineAnnual = baselineAnnual.rename(columns={"Scenario":"B_Scenario",
                              "Date_Time":"Date_Time",
                             "Illuminance(lux)":"B_Illuminance(lux)",
                             "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                             "HeatingEnergy(kBtu)":"B_HeatingEnergy(kBtu)",
                             "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                             "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
                             "FanCost($)":"B_FanEnergy($)",
                             "HeatingCost($)":"B_HeatingCost($)",
                             "CoolingCost($)":"B_CoolingCost($)",
                             "AnnualCost($)":"B_AnnualCost($)",
                             })

#allBaselineHourly

In [21]:
# Merge the dataframes to include the baseline data.
AnnualWithBaseline = pd.merge(allAnnualData, allBaselineAnnual, on="Date_Time", how="left")

#hourlyWithBaseline

In [22]:
#list(AnnualWithBaseline.columns.values)

In [23]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaseline["deltaEnergy(kBtu)"] = AnnualWithBaseline["AnnualEnergy(kBtu)"] - AnnualWithBaseline["B_AnnualEnergy(kBtu)"]
AnnualWithBaseline["deltaCost($)"] = AnnualWithBaseline["AnnualCost($)"] - AnnualWithBaseline["B_AnnualCost($)"]

# AnnualWithBaseline

In [24]:
# Convert NANs to 0
AnnualWithBaseline =AnnualWithBaseline.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [25]:
# Group by scenario
annualData = AnnualWithBaseline.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [26]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [27]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [28]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [29]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [30]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [31]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [32]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanCost($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingCost($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingCost($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [33]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [34]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner")

#allMonthlyData

In [35]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanCost($)"] + allMonthlyData["HeatingCost($)"] + allMonthlyData["CoolingCost($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [36]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($),Month
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.215039,01
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.234931,01
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.251555,01
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267499,01
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136,01
...,...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.004310,12
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.053998,12


## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [37]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData2.sum()

In [38]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [39]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [40]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [41]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [42]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [43]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [44]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [45]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [46]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [47]:
# Convert Joules to kBtus in new dataframe
get_hourly["FanEnergy(kBtu)"] = get_hourly["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly["HeatingEnergy(kBtu)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly["CoolingEnergy(kBtu)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly

In [48]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly

In [49]:
#list(get_hourly.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [50]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [51]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [52]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanCost($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingCost($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingCost($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [53]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [54]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [55]:
#Extract hour, day, month.
allHourlyData["Month"] = allHourlyData.Date_Time.str.slice(0,3)
allHourlyData["Day"] = allHourlyData.Date_Time.str.slice(4,6)
#allHourlyData["Hour"] = allHourlyData.Date_Time.str.slice(8,16)
allHourlyData["Hour"] = allHourlyData["Date_Time"].astype(str).str[8:16]

allHourlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanCost($),HeatingCost($),CoolingCost($),Month,Day,Hour
0,Clear/Clear,01/01 01:00:00,0.0,0.580415,17.656572,0.0,0.019052,0.195988,0.0,01,01,01:00:00
1,Clear/Clear,01/01 02:00:00,0.0,0.634104,19.289833,0.0,0.020814,0.214117,0.0,01,01,02:00:00
2,Clear/Clear,01/01 03:00:00,0.0,0.678973,20.654778,0.0,0.022287,0.229268,0.0,01,01,03:00:00
3,Clear/Clear,01/01 04:00:00,0.0,0.722009,21.963976,0.0,0.023699,0.243800,0.0,01,01,04:00:00
4,Clear/Clear,01/01 05:00:00,0.0,0.742622,22.591009,0.0,0.024376,0.250760,0.0,01,01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,12,31,20:00:00
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,12,31,21:00:00
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.011634,0.353927,0.0,0.000382,0.003929,0.0,12,31,22:00:00
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.145747,4.433728,0.0,0.004784,0.049214,0.0,12,31,23:00:00


In [56]:
# Add energy and cost totals

totalEnergyhourly = allHourlyData["FanEnergy(kBtu)"] + allHourlyData["HeatingEnergy(kBtu)"] + allHourlyData["CoolingEnergy(kBtu)"]
totalCosthourly = allHourlyData["FanCost($)"] + allHourlyData["HeatingCost($)"] + allHourlyData["CoolingCost($)"]
allHourlyData["HourlyEnergy(kBtu)"] = totalEnergyhourly
allHourlyData["HourlyCost($)"] = totalCosthourly

#allHourlyData

In [57]:
#list(allHourlyData.columns.values)

---

## Establish Hourly Baseline at Clear/Clear-VB0

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Filter the data on the conditionals of illuminance and energy performance.

In [58]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineHourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baselineHourly

In [59]:
# Rename columns to indicate baseline to prepare for merge

allBaselineHourly = baselineHourly.rename(columns={"Scenario":"B_Scenario",
                              "Date_Time":"Date_Time",
                             "Illuminance(lux)":"B_Illuminance(lux)",
                             "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                             "HeatingEnergy(kBtu)":"B_HeatingEnergy(kBtu)",
                             "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                             "HourlyEnergy(kBtu)":"B_HourlyEnergy(kBtu)",
                             "FanCost($)":"B_FanEnergy($)",
                             "HeatingCost($)":"B_HeatingCost($)",
                             "CoolingCost($)":"B_CoolingCost($)",
                             "HourlyCost($)":"B_HourlyCost($)",
                             "Month":"B_Month",
                             "Day":"B_Day",
                             "Hour":"B_Hour"
                             })

#allBaselineHourly

In [60]:
# Merge the dataframes to include the baseline data.
hourlyWithBaseline = pd.merge(allHourlyData, allBaselineHourly, on="Date_Time", how="left")

#hourlyWithBaseline

In [61]:
#list(hourlyWithBaseline.columns.values)

In [62]:
# Add column for the difference from baseline columns (scenario - baseline)
hourlyWithBaseline["deltaEnergy(kBtu)"] = hourlyWithBaseline["HourlyEnergy(kBtu)"] - hourlyWithBaseline["B_HourlyEnergy(kBtu)"]
hourlyWithBaseline["deltaCost($)"] = hourlyWithBaseline["HourlyCost($)"] - hourlyWithBaseline["B_HourlyCost($)"]
hourlyWithBaseline["%deltaEnergy(kBtu)"] = (hourlyWithBaseline["HourlyEnergy(kBtu)"] - hourlyWithBaseline["B_HourlyEnergy(kBtu)"]) / hourlyWithBaseline["B_HourlyEnergy(kBtu)"].replace({0:np.nan}) 
hourlyWithBaseline["%deltaCost($)"] = (hourlyWithBaseline["HourlyCost($)"] - hourlyWithBaseline["B_HourlyCost($)"]) / hourlyWithBaseline["B_HourlyCost($)"].replace({0:np.nan}) 

#hourlyWithBaseline

In [63]:
# Convert NANs to 0
hourlyWithBaseline =hourlyWithBaseline.fillna(0)

#hourlyWithBaseline

In [64]:
# Export hourly data with baseline difference file to csv
hourlyWithBaseline.to_csv("../Scraper_Output/hou_hourly_data.csv", header=True, index=True)

---

## Add conditionals

Illumination >= 300.<br> 
EnergyPerformance <= selected percentage.

In [65]:
conditionals = hourlyWithBaseline[hourlyWithBaseline["Illuminance(lux)"] >= 300]

conditionals

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanCost($),HeatingCost($),CoolingCost($),Month,...,B_CoolingCost($),B_Month,B_Day,B_Hour,B_HourlyEnergy(kBtu),B_HourlyCost($),deltaEnergy(kBtu),deltaCost($),%deltaEnergy(kBtu),%deltaCost($)
8,Clear/Clear,01/01 09:00:00,916.110486,0.382079,11.623076,0.00000,0.012541,0.129016,0.000000,01,...,0.00000,01,01,09:00:00,11.917324,0.140522,0.087830,0.001036,0.007370,0.007370
9,Clear/Clear,01/01 10:00:00,1252.311322,0.047685,1.450606,0.00000,0.001565,0.016102,0.000000,01,...,0.00000,01,01,10:00:00,2.096148,0.024717,-0.597857,-0.007050,-0.285217,-0.285217
10,Clear/Clear,01/01 11:00:00,2064.821629,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,01,...,0.00000,01,01,11:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11,Clear/Clear,01/01 12:00:00,26693.640010,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,01,...,0.00000,01,01,12:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,Clear/Clear,01/01 13:00:00,44152.458090,0.085454,0.000000,0.46433,0.002805,0.000000,0.015241,01,...,0.00000,01,01,13:00:00,0.000000,0.000000,0.549784,0.018046,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429155,lowE(2)/Clear-BBG60,12/28 12:00:00,304.374930,0.162792,4.952231,0.00000,0.005343,0.054970,0.000000,12,...,0.00000,12,28,12:00:00,0.000000,0.000000,5.115023,0.060313,0.000000,0.000000
429203,lowE(2)/Clear-BBG60,12/30 12:00:00,308.241751,0.130722,3.976636,0.00000,0.004291,0.044141,0.000000,12,...,0.00000,12,30,12:00:00,0.000000,0.000000,4.107358,0.048431,0.000000,0.000000
429226,lowE(2)/Clear-BBG60,12/31 11:00:00,306.507098,0.187002,5.688718,0.00000,0.006138,0.063145,0.000000,12,...,0.00000,12,31,11:00:00,0.000000,0.000000,5.875720,0.069283,0.000000,0.000000
429227,lowE(2)/Clear-BBG60,12/31 12:00:00,315.096196,0.054647,1.662409,0.00000,0.001794,0.018453,0.000000,12,...,0.00000,12,31,12:00:00,0.000000,0.000000,1.717057,0.020246,0.000000,0.000000


In [66]:
# # Create datasets that show reduction in energy and cost 10, 20 and 30%

# energy10 = conditionals[conditionals["HourlyEnergy(kBtu)"] <= (conditionals["B_HourlyEnergy(kBtu)"] * .9)]
# energy20 = conditionals[conditionals["HourlyEnergy(kBtu)"] <= (conditionals["B_HourlyEnergy(kBtu)"] * .8)]
# energy30 = conditionals[conditionals["HourlyEnergy(kBtu)"] <= (conditionals["B_HourlyEnergy(kBtu)"] * .7)]
# energy40 = conditionals[conditionals["HourlyEnergy(kBtu)"] <= (conditionals["B_HourlyEnergy(kBtu)"] * .6)]
# cost10 = conditionals[conditionals["HourlyCost($)"] <= (conditionals["B_HourlyCost($)"] * .9)]
# cost20 = conditionals[conditionals["HourlyCost($)"] <= (conditionals["B_HourlyCost($)"] * .8)]
# cost30 = conditionals[conditionals["HourlyCost($)"] <= (conditionals["B_HourlyCost($)"] * .7)]
# cost40 = conditionals[conditionals["HourlyCost($)"] <= (conditionals["B_HourlyCost($)"] * .6)]

In [67]:
# Export hourly conditional data with to csvs
conditionals.to_csv("../Scraper_Output/hou_conditionals_data.csv", header=True, index=True)
# energy10.to_csv("../Scraper_Output/hou_energy10_data.csv", header=True, index=True)
# energy20.to_csv("../Scraper_Output/hou_energy20_data.csv", header=True, index=True)
# energy30.to_csv("../Scraper_Output/hou_energy30_data.csv", header=True, index=True)
# energy40.to_csv("../Scraper_Output/hou_energy40_data.csv", header=True, index=True)
# cost10.to_csv("../Scraper_Output/hou_cost10_data.csv", header=True, index=True)
# cost20.to_csv("../Scraper_Output/hou_cost20_data.csv", header=True, index=True)
# cost30.to_csv("../Scraper_Output/hou_cost30_data.csv", header=True, index=True)
# cost40.to_csv("../Scraper_Output/hou_cost40_data.csv", header=True, index=True)